<a href="https://colab.research.google.com/github/Cirediallo/Models/blob/main/icefall_asr_librispeech_pretrained_tdnn_lstm_ctc_usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Description
This notebook shows how to use a pre-trained tdnn-lstm_ctc model with [icefall](https://github.com/k2-fsa/icefall/).

## Environment setup

To use a pre-trained model with icefall, we have to install the following dependencies:

- [k2][k2], for FSA operations
- [torchaudio][audio], for reading sound files
- [kaldifeat][kaldifeat], for extracting features from a single sound
  file or multiple sound files

**NOTE**: [lhotse][lhotse] is used only in training time, for data preparation.


[k2]: https://github.com/k2-fsa/k2
[audio]: https://github.com/pytorch/audio
[kaldifeat]: https://github.com/csukuangfj/kaldifeat
[lhotse]: https://github.com/lhotse-speech/lhotse

### Install PyTorch and torchaudio

In [1]:
! nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


Colab provides CUDA 11.0, so we choose to install torch==1.7.1+cu110.

The reason to select torch 1.7.1 is that we are going to install k2 using `pip install`, which depends on torch 1.7.1.

If you want to use a different version of PyTorch, please
refer to <https://k2.readthedocs.io/en/latest/installation/index.html> for installing k2 either from source or with `conda install`.

In [21]:
#! pip install torch==1.7.1+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

### Install k2

In [22]:
#! pip install k2==1.17

In [12]:
#! pip install k2==1.17.dev20220725+cuda11.0.torch1.7.1 -f https://k2-fsa.org/nightly/index.html

Check that k2 was installed successfully:

In [23]:
#! python3 -m k2.version

### All previous steps in one

In [2]:
!pip install torch==1.13.1
!pip install torchaudio==0.13.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.2 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cu118
    Uninstalling torch-2.0.0+cu118:
      Successfully uninstalled torch-2.0.0+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.15.1+cu118 requires torch==2.0.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.15.1 requires torch==2.0.0, but you have torch

In [3]:
pip install k2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 MB 10.7 MB/s eta 0:00:00


In [4]:
! python3 -m k2.version


k2 version: 1.23.4
Build type: Release
Git SHA1: 62e404dd3f3a811d73e424199b3408e309c06e1a
Git date: Mon Jan 30 02:26:16 2023
Cuda used to build k2: 11.7
cuDNN used to build k2: 8.2.0
Python version used to build k2: 3.9
OS used to build k2: Ubuntu 18.04.6 LTS
CMake version: 3.25.1
GCC version: 7.5.0
CMAKE_CUDA_FLAGS:  -Wno-deprecated-gpu-targets   -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_35,code=sm_35  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_50,code=sm_50  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_60,code=sm_60  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_61,code=sm_61  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_70,code=sm_70  -lineinfo --expt-extended-lambda -use_fast_math -

### Install kaldifeat

In [5]:
! pip install kaldifeat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 482.4/482.4 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaldifeat: filename=kaldifeat-1.21-cp39-cp39-linux_x86_64.whl size=263529 sha256=f9d7c27e98071ad5d4b0452307cff00fe952edc09bc1db5f9ac3132a6574ac55
  Stored in directory: /root/.cache/pip/wheels/d0/db/70/915de4b2e80a9aa90097d52e978ec5eb5c7393c1ee32f2618c
Successfully built kaldifeat


To check that kaldifeat was installed successfully, run

In [6]:
! python3 -c "import kaldifeat; print(kaldifeat.__version__)"

1.21


### Install icefall

icefall is a collection of Python scripts. All you need is just to
download its source code and set the `PYTHONPATH` environment variable.

In [7]:
! git clone https://github.com/k2-fsa/icefall

Cloning into 'icefall'...
remote: Enumerating objects: 11702, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 11702 (delta 26), reused 40 (delta 10), pack-reused 11627
Receiving objects: 100% (11702/11702), 13.61 MiB | 28.11 MiB/s, done.
Resolving deltas: 100% (7973/7973), done.


In [8]:
! pip install -q kaldialign

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 3.6 MB/s eta 0:00:00


### install lhoste

In [9]:
! pip install git+https://github.com/lhotse-speech/lhotse

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/lhotse-speech/lhotse to /tmp/pip-req-build-t_1mtvqt
  Running command git clone --filter=blob:none --quiet https://github.com/lhotse-speech/lhotse /tmp/pip-req-build-t_1mtvqt
  Resolved https://github.com/lhotse-speech/lhotse to commit 0f812851aefb1dc0560e76641f86bbcfcd96a47c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 37.3 MB/s eta 0:00:00
  Created wheel for lhotse: filename=lhotse-1.14.0.dev0+git.0f81285.clean-py3-none-any.whl size=636039 sha256=2bc6710a3fa7c74ed2f95e716052885f114ced829dd5f575c3849cdb0bac7039
  Stored in directory: /tmp/pip-ephem-wheel-cache-54o894h1/wheels/ed/a4/6e/dc8c64698c

In [1]:
! cd icefall && \
  ! pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.9 MB/s eta 0:00:00


## Download pre-trained Tdnn-Lstm CTC model

To make the following steps easier, we decide to download the model
to `icefall/egs/librispeech/ASR/tmp-lstm`

In [2]:
! apt-get install -y -qq tree sox git-lfs

Selecting previously unselected package libopencore-amrnb0:amd64.
(Reading database ... 122349 files and directories currently installed.)
Preparing to unpack .../0-libopencore-amrnb0_0.1.5-1_amd64.deb ...
Unpacking libopencore-amrnb0:amd64 (0.1.5-1) ...
Selecting previously unselected package libopencore-amrwb0:amd64.
Preparing to unpack .../1-libopencore-amrwb0_0.1.5-1_amd64.deb ...
Unpacking libopencore-amrwb0:amd64 (0.1.5-1) ...
Selecting previously unselected package libsox3:amd64.
Preparing to unpack .../2-libsox3_14.4.2+git20190427-2+deb11u2build0.20.04.1_amd64.deb ...
Unpacking libsox3:amd64 (14.4.2+git20190427-2+deb11u2build0.20.04.1) ...
Selecting previously unselected package libsox-fmt-alsa:amd64.
Preparing to unpack .../3-libsox-fmt-alsa_14.4.2+git20190427-2+deb11u2build0.20.04.1_amd64.deb ...
Unpacking libsox-fmt-alsa:amd64 (14.4.2+git20190427-2+deb11u2build0.20.04.1) ...
Selecting previously unselected package libsox-fmt-base:amd64.
Preparing to unpack .../4-libsox-fmt-b

**CAUTION**: You have to run `sudo apt-get install git-lfs`
and `git lfs install` in order to download the pre-trained model.

In [ ]:
! cd icefall/egs/librispeech/ASR && rm -rf tmp-lstm

In [ ]:
! cd icefall/egs/librispeech/ASR && \
  mkdir tmp-lstm && \
  cd tmp-lstm && \
  git lfs install && \
  git clone https://huggingface.co/pkufool/icefall_asr_librispeech_tdnn-lstm_ctc && \
  cd icefall_asr_librispeech_tdnn-lstm_ctc && \
  cd ../.. && \
  tree tmp-lstm

In [ ]:
! ffprobe -show_format icefall/egs/librispeech/ASR/tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/test_wavs/1089-134686-0001.flac

In [ ]:
! ffprobe -show_format icefall/egs/librispeech/ASR/tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/test_wavs/1221-135766-0001.flac

In [ ]:
! ffprobe -show_format icefall/egs/librispeech/ASR/tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/test_wavs/1221-135766-0002.flac

Due to limited memory provided by Colab, you have to upgrade to Colab Pro
to run `HLG decoding + LM rescoring`.

In [ ]:
! pip install lhotse sentencepiece

## HLG decoding (1best)

In [ ]:
! cd icefall/egs/librispeech/ASR && \
    PYTHONPATH=/content/icefall python3 ./tdnn_lstm_ctc/pretrained.py \
      --method 1best \
      --checkpoint ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/exp/pretrained.pt \
      --words-file ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/data/lang_phone/words.txt \
      --HLG ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/data/lang_phone/HLG.pt \
      ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/test_wavs/1089-134686-0001.flac \
      ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/test_wavs/1221-135766-0001.flac \
      ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/test_wavs/1221-135766-0002.flac 

/usr/local/lib/python3.7/dist-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '
2022-11-02 14:32:26,910 INFO [pretrained.py:162] {'feature_dim': 80, 'subsampling_factor': 3, 'num_classes': 72, 'sample_rate': 16000, 'search_beam': 20, 'output_beam': 5, 'min_active_states': 30, 'max_active_states': 10000, 'use_double_scores': True, 'checkpoint': './tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/exp/pretrained.pt', 'words_file': './tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/data/lang_phone/words.txt', 'HLG': './tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/data/lang_phone/HLG.pt', 'method': '1best', 'G': None, 'ngram_lm_scale': 0.8, 'sound_files': ['./tmp-lstm/icefall_asr_librispeech_tdnn-

## HLG decoding + LM rescoring

In [ ]:
! cd icefall/egs/librispeech/ASR && \
    PYTHONPATH=/content/icefall python3 ./tdnn_lstm_ctc/pretrained.py \
      --method whole-lattice-rescoring \
      --checkpoint ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/exp/pretrained.pt \
      --words-file ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/data/lang_phone/words.txt \
      --HLG ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/data/lang_phone/HLG.pt \
      --G ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/data/lm/G_4_gram.pt \
      --ngram-lm-scale 0.08 \
      ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/test_wavs/1089-134686-0001.flac \
      ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/test_wavs/1221-135766-0001.flac \
      ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/test_wavs/1221-135766-0002.flac 

/usr/local/lib/python3.7/dist-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '
2022-11-02 14:35:30,881 INFO [pretrained.py:162] {'feature_dim': 80, 'subsampling_factor': 3, 'num_classes': 72, 'sample_rate': 16000, 'search_beam': 20, 'output_beam': 5, 'min_active_states': 30, 'max_active_states': 10000, 'use_double_scores': True, 'checkpoint': './tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/exp/pretrained.pt', 'words_file': './tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/data/lang_phone/words.txt', 'HLG': './tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/data/lang_phone/HLG.pt', 'method': 'whole-lattice-rescoring', 'G': './tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/data/lm/G_4_gram.p

## Training

### Data preparation

In [3]:
! cd icefall/egs/librispeech/ASR && \
  ./prepare.sh

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Traceback (most recent call last):
  File "/usr/local/bin/lhotse", line 8, in <module>
    sys.exit(cli())
  File "/usr/local/lib/python3.9/dist-packages/click/core.py", line 1130, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.9/dist-packages/click/core.py", line 1055, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.9/dist-packages/click/core.py", line 1657, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/usr/local/lib/python3.9/dist-packages/click/core.py", line 1657, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/usr/local/lib/python3.9/dist-packages/click/core.py", line 1404, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.9/dist-packages/click/core.py", line 760, in invoke
    return __callback(*args, **kwargs)
  File "/usr/local/lib/python3.9/dist-

### Launch training

In [ ]:
# https://k2-fsa.github.io/icefall/recipes/Non-streaming-ASR/librispeech/tdnn_lstm_ctc.html